In [1]:
!pip install langchain_openai

In [2]:
!pip install crewai
!pip install 'crewai[tools]'
!pip install langchain-community
!pip install langchain_groq

In [ ]:
!pip install python-dotenv

In [ ]:
import os
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process
from langchain_community.llms import Ollama
from langchain_groq import ChatGroq
from langchain_community.llms import HuggingFaceEndpoint

In [ ]:
from langchain_openai import ChatOpenAI
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

In [11]:
# from langchain.chat_models import ChatOpenAI

# llm = ChatOpenAI(
#     model_name="gpt-3.5-turbo",
#     openai_api_key="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"
# )


<ipython-input-11-79496d44b6d7>:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [6]:
financial_analyst = Agent(
    role="Financial Analyst",
    goal="Analyze income, expenses, and suggest optimal budget allocation",
    backstory="""You are an experienced financial analyst with expertise in personal
    budgeting. You help individuals understand their spending patterns and create
    realistic budgets.""",
    verbose=True,
    llm=llm
)

budget_analysis_task = Task(
    description="""Analyze the user's monthly income and expenses. Categorize spending
    into essential (housing, utilities, food) and non-essential categories.
    Identify areas where spending can be reduced.

    Example Input:
    - Monthly income: $4,000
    - Rent: $1,200
    - Utilities: $200
    - Groceries: $500
    - Eating out: $400
    - Entertainment: $300
    - Car payment: $350
    - Insurance: $150
    - Miscellaneous: $300
    """,
    agent=financial_analyst,
    expected_output="""A detailed breakdown of current spending patterns with
    recommendations for budget adjustments."""
)

In [7]:
investment_advisor = Agent(
    role="Investment Advisor",
    goal="Provide investment recommendations based on financial goals and risk profile",
    backstory="""You are a seasoned investment advisor who helps people grow their
    wealth through smart investment choices. You consider risk tolerance, time
    horizon, and financial goals when making recommendations.""",
    verbose=True,
    llm=llm
)

investment_recommendation_task = Task(
    description="""Based on the user's financial goals, risk tolerance, and time horizon,
    recommend suitable investment options.

    Example Input:
    - Financial goals: Retirement in 30 years, buying a house in 5 years
    - Risk tolerance: Moderate
    - Current savings: $20,000
    - Monthly contribution capacity: $500
    """,
    agent=investment_advisor,
    expected_output="""Investment recommendations including asset allocation suggestions
    and specific investment vehicles appropriate for their goals."""
)

In [8]:
debt_manager = Agent(
    role="Debt Management Specialist",
    goal="Create debt repayment strategies that minimize interest and maximize savings",
    backstory="""You are a debt management expert who has helped hundreds of people
    get out of debt. You know how to prioritize debt repayment and negotiate with
    creditors when necessary.""",
    verbose=True,
    llm=llm
)

debt_repayment_task = Task(
    description="""Create a debt repayment plan that prioritizes high-interest debt
    while maintaining minimum payments on all debts.

    Example Input:
    - Credit card debt: $8,000 (18% APR)
    - Student loans: $15,000 (5% APR)
    - Car loan: $10,000 (6% APR)
    - Available for debt repayment: $1,000/month
    """,
    agent=debt_manager,
    expected_output="""A step-by-step debt repayment plan with timeline and strategies
    to reduce interest payments."""
)

In [9]:
savings_coach = Agent(
    role="Savings Coach",
    goal="Develop savings strategies for emergency funds and future financial goals",
    backstory="""You are a savings coach who motivates people to build healthy savings habits.
    You help individuals establish emergency funds and save for important life goals.""",
    verbose=True,
    llm=llm
)

savings_plan_task = Task(
    description="""Develop a savings strategy to build an emergency fund and save for
    future goals while accounting for current debt repayment obligations.

    Example Input:
    - Current savings: $2,000
    - Monthly income after expenses: $800
    - Short-term goal: $10,000 emergency fund
    - Long-term goal: $30,000 for house down payment in 5 years
    """,
    agent=savings_coach,
    expected_output="""A savings plan with specific monthly targets and strategies
    to reach both short-term and long-term savings goals."""
)

In [15]:
personal_finance_crew = Crew(
    agents=[financial_analyst, investment_advisor, debt_manager, savings_coach],
    tasks=[budget_analysis_task, investment_recommendation_task, debt_repayment_task, savings_plan_task],
    verbose=True,
    process=Process.sequential  # Tasks will be executed one after another
)

# Main function to execute the crew
def analyze_personal_finances(user_info):
    """
    Execute the personal finance advisory crew with the user's financial information.

    Args:
        user_info (dict): Dictionary containing user's financial information

    Returns:
        dict: Financial advice from all the agents
    """
    print("Starting personal finance analysis...")

    # Adapt the tasks based on user info
    for task in personal_finance_crew.tasks:
        task.context = f"User Information: {user_info}"

    # Catch any potential errors
    try:
        # Execute the crew
        result = personal_finance_crew.kickoff()
        return result
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        print("Check if your LLM is properly configured and running.")
        return {"error": str(e)}

# Example usage
if __name__ == "__main__":
    # Example user information
    user_info = {
        "monthly_income": 4000,
        "expenses": {
            "rent": 1200,
            "utilities": 200,
            "groceries": 500,
            "eating_out": 400,
            "entertainment": 300,
            "car_payment": 350,
            "insurance": 150,
            "miscellaneous": 300
        },
        "debts": {
            "credit_card": {"balance": 8000, "apr": 18},
            "student_loans": {"balance": 15000, "apr": 5},
            "car_loan": {"balance": 10000, "apr": 6}
        },
        "savings": 2000,
        "financial_goals": {
            "short_term": "emergency_fund",
            "long_term": ["house_down_payment", "retirement"],
            "time_horizon": {"emergency_fund": 1, "house_down_payment": 5, "retirement": 30}
        },
        "risk_tolerance": "moderate"
    }

    financial_advice = analyze_personal_finances(user_info)
    print("\nFinancial Advice Summary:")
    print(financial_advice)

Starting personal finance analysis...


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 8976e122-8325-49c9-a59c-40b35d7aecee                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 2ef465ad-238a-4146-92c2-06506792c2f8
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 2ef465ad-238a-4146-92c2-06506792c2f8
       Status: Executing Task...
    └── 🤖 Agent: Financial Analyst
            Status: In Progress

# Agent: Financial Analyst
## Task: Analyze the user's monthly income and expenses. Categorize spending 
    into essential (housing, utilities, food) and non-essential categories. 
    Identify areas where spending can be reduced.
    
    Example Input:
    - Monthly income: $4,000
    - Rent: $1,200
    - Utilities: $200
    - Groceries: $500
    - Eating out: $400
    - Entertainment: $300
    - Car payment: $350
    - Insurance: $150
    - Miscellaneous: $300
    


# Agent: Financial Analyst
## Final Answer: 
**Income and Expense Analysis**

**Monthly Income:**  
- Total Monthly Income: $4,000

**Monthly Expenses Breakdown:**  
1. **Essential Expenses:**  
   - Rent: $1,200  
   - Utilities: $200  
   - Groceries: $500  
   - Car Payment: $350  
   - Insurance: $150  
   - **Total Essential Expenses:** $2,600  

2. **Non-Essential Expenses:**  
   - Eating Out: $400  
   - Entertainment: $300  
   - Miscellaneous: $300  
   - **Total Non-Essential Expenses:** $1,000  

**Total Mo

...

🚀 Crew: crew
├── 📋 Task: 2ef465ad-238a-4146-92c2-06506792c2f8
│      Assigned to: Financial Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: Financial Analyst
│           Status: ✅ Completed
├── 📋 Task: 86d6cad8-378d-496b-9a37-4944c818074d
│      Assigned to: Investment Advisor
│      Status: ✅ Completed
│   └── 🤖 Agent: Investment Advisor
│           Status: ✅ Completed
├── 📋 Task: 0765ac84-93d3-4b07-9db2-a5a6698a0210
│      Assigned to: Debt Management Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Debt Management Specialist
│           Status: ✅ Completed
└── 📋 Task: efa82c20-184a-4318-bfdb-93c1759c1979
       Status: Executing Task...
    └── 🤖 Agent: Savings Coach
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 2ef465ad-238a-4146-92c2-06506792c2f8
│      Assigned to: Financial Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: Financial Analyst
│           Status: ✅ Completed
├── 📋 Task: 86d6cad8-378d-496b-9a37-4944c818074d
│      Assigned to: Investment Advisor
│      Status: ✅ Completed
│   └── 🤖 Agent: Investment Advisor
│           Status: ✅ Completed
├── 📋 Task: 0765ac84-93d3-4b07-9db2-a5a6698a0210
│      Assigned to: Debt Management Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Debt Management Specialist
│           Status: ✅ Completed
└── 📋 Task: efa82c20-184a-4318-bfdb-93c1759c1979
       Assigned to: Savings Coach
       Status: ✅ Completed
    └── 🤖 Agent: Savings Coach
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: efa82c20-184a-4318-bfdb-93c1759c1979                                                                     │
│  Agent: Savings Coach                                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 8976e122-8325-49c9-a59c-40b35d7aecee                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


Financial Advice Summary:
To successfully build an emergency fund while also saving for a house down payment, we can create a detailed savings strategy that accounts for your current financial situation and obligations. Here’s a step-by-step plan:

1. **Emergency Fund Goal**: Your target is $10,000. You currently have $2,000 saved, which means you need to save an additional $8,000.

2. **Time Frame for Emergency Fund**: Let’s set a timeline of 1 year (12 months) to reach your emergency fund goal. 

3. **Monthly Savings Target for Emergency Fund**: 
   - Required Amount: $8,000
   - Time Frame: 12 months
   - Monthly Savings Target = $8,000 / 12 months = **$667 per month**.

4. **Long-term Goal**: You want to save $30,000 for a house down payment in 5 years (60 months).

5. **Time Frame for House Down Payment**: 
   - You can start saving for this once your emergency fund is fully funded. However, if you can manage to save for both simultaneously, we will explore that option.

6. **Mon

In [17]:
!pip install IPython

In [20]:
from IPython.display import display, Markdown

In [22]:
display(Markdown(financial_advice.raw))

To successfully build an emergency fund while also saving for a house down payment, we can create a detailed savings strategy that accounts for your current financial situation and obligations. Here’s a step-by-step plan:

1. **Emergency Fund Goal**: Your target is $10,000. You currently have $2,000 saved, which means you need to save an additional $8,000.

2. **Time Frame for Emergency Fund**: Let’s set a timeline of 1 year (12 months) to reach your emergency fund goal. 

3. **Monthly Savings Target for Emergency Fund**: 
   - Required Amount: $8,000
   - Time Frame: 12 months
   - Monthly Savings Target = $8,000 / 12 months = **$667 per month**.

4. **Long-term Goal**: You want to save $30,000 for a house down payment in 5 years (60 months).

5. **Time Frame for House Down Payment**: 
   - You can start saving for this once your emergency fund is fully funded. However, if you can manage to save for both simultaneously, we will explore that option.

6. **Monthly Savings Target for House Down Payment**:
   - Required Amount: $30,000
   - Time Frame: 60 months
   - Monthly Savings Target = $30,000 / 60 months = **$500 per month**.

7. **Total Monthly Savings Requirement**:
   - If you focus solely on the emergency fund first, you would need $667/month for the first year.
   - After completing your emergency fund, you can then allocate $500/month towards the house down payment.

8. **Strategic Approach**:
   - **First Year Plan**: 
     - Allocate $667 from your monthly income of $800 towards the emergency fund.
     - This will leave you with $133 for other expenses or debt repayment.
   
   - **Second Year Plan (Post Emergency Fund Completion)**:
     - Once the emergency fund is fully funded, redirect the $667 previously allocated to the emergency fund towards the house down payment.
     - Along with the $500/month you planned for the house, you will then be saving a total of $1,167/month for the down payment.

9. **Debt Considerations**: 
   - Ensure that your current debt repayment obligations are being met with your remaining $133/month during the emergency fund saving period. Adjust your budget to make sure you’re not falling behind on debt payments.
   - If your debt payments exceed $133, consider prioritizing higher interest debts or finding ways to reduce expenses to free up additional funds.

10. **Additional Tips**:
    - Automate your savings: Set up automatic transfers to your savings accounts right after you receive your paycheck.
    - Look for extra income opportunities: Consider side jobs or freelance work to increase your monthly income, which can help you reach your savings goals faster.
    - Review and adjust: Regularly reassess your budget and savings progress. If your income increases or expenses decrease, consider increasing your savings contributions.

By following this structured approach, you will be on track to build a solid emergency fund and save for your future home, setting yourself up for a more secure financial future.